In [ ]:
!pip install rdkit
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
import seaborn as sns
import math
from joblib import dump, load
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
import math

In [76]:
df1 = pd.read_csv('/content/drive/MyDrive/generations/1.csv')
df1 = df1.drop(columns=['Unnamed: 0'])

In [77]:
df2 = pd.read_csv('/content/drive/MyDrive/generations/2.csv')
df2 = df2.drop(columns=['Unnamed: 0'])

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/generations/3.csv')
df3 = df3.drop(columns=['Unnamed: 0'])

In [78]:
df4 = pd.read_csv('/content/drive/MyDrive/generations/4.csv')
df4 = df4.drop(columns=['Unnamed: 0'])

In [79]:
df = pd.concat([df1, df2, df4], axis=0)

In [80]:
df['smiles']

0                            O=P
1                            O=C
2                            O=N
3                            O=S
4                       O=[C@@H]
                  ...           
8165      c1cccc(C(=O)N(CC)CC)n1
8166      CCN(CC)C(=O)C(=O)N(CC)
8167                c1cccc(CC)n1
8168    CCN(CC)C(=O)c1cccc(CC)n1
8169      c1cccc(C(=O)N(CC)CC)n1
Name: smiles, Length: 59731, dtype: object

In [81]:
df['mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))

In [82]:
df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))
df['num_rings'] = df['mol'].apply(lambda x: Descriptors.RingCount(x))
df['Fp_Density_Morgan1'] = df['mol'].apply(lambda x: Descriptors.FpDensityMorgan1(x))
df['HeavyAtomMolWt'] = df['mol'].apply(lambda x: Descriptors.HeavyAtomMolWt(x))

In [83]:
def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

symbols = ["C", "O", "N"]
number_of_atoms(symbols, df)

In [84]:
cb = load('/content/drive/MyDrive/models/cb.joblib')
rf = load('/content/drive/MyDrive/models/rf.joblib')
lr = load('/content/drive/MyDrive/models/lr.joblib')

In [85]:
df_ml = df.drop(columns=['smiles', 'mol'])

In [86]:
predict_cb = cb.predict(df_ml)
predict_rf = rf.predict(df_ml)
predict_lr = lr.predict(df_ml)

In [87]:
df['cb'] = predict_cb
df['rf'] = predict_rf
df['lr'] = predict_lr

In [88]:
df['lgK'] = (df['cb'] + df['rf'] + df['lr'])/3

In [89]:
sub = pd.DataFrame(df.sort_values(by='lgK', ascending=False)['smiles'])

In [90]:
sub.to_csv('gener.csv', header=False, index=False)